# Install Packages

In [ ]:
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken
!pip install -U langchain-community
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.7/336.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

# Import Libraries

In [ ]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain, LLMChain, StuffDocumentsChain
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from typing_extensions import Concatenate

In [ ]:
#Load pdf

pdfreader = PdfReader('/path/to.pdf')

##If you are using drive

# from google.colab import drive
# drive.mount('/content/drive')
# pdfreader = PdfReader('/path/to/file.pdf')

In [ ]:
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [ ]:
raw_text

' \n \nPart 3 – DrillsFront shoulder opens up too quickly (trouble reaching the outside pitch) Drills: 2, 3, 8, 15  Uppercut - Drills: 4, 6, 7, 8, 13, 14Wrists roll over too quickly (Hitting too many weak ground balls to pull side) - Drills: 2, 3, 4, 5, 7, 8, 9, 10, 13, 14, 15, 18, 20Inconsistent hand path (Not hitting many balls on the barrel) - Drills: 1, 4, 8, 9, 10, 12, 17, 19Backside collapses or not getting on back toe at contact (Fouling good pitches straight back); not getting to 50/50 weight distribution - Drills: 3, 5, 6, 7, 8, 9, 13, 14Weak swing (i.e. Looking to hit with more power/bat speed) - Drills: 2, 3, 7, 9, 11, 16, 20Too stiff (no rhythm) - Drills: 6, 9, 10, 18, 19Strike zone judgment - Drills: 1, 12, 13Too many swing and misses Drills: 1, 2, 4, 8, 9, 15, 17, 19No separation at contact (Your hands come with you when you stride. They don’t stay back. Weak pop flies to the opposite field) - Drills: 3, 5, 7, 12, 17, 19Upper body dominant (Have trouble with low pitch, do

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000,  # Adjust this size according to your needs
    chunk_overlap=200,  # Overlap to ensure context is maintained
    length_function=len,
    separators=["\n\n", "\n", ".", " "]  # Add multiple separators
)

# Split the text into chunks
texts = text_splitter.split_text(raw_text)

In [ ]:
len(texts)

16

# Set Up OpenAI API Key

In [ ]:
openai_api_key = "YOUR API KEY"   # Replace your OpenAI API key here.

In [ ]:
llm = OpenAI(api_key=openai_api_key)
embeddings = OpenAIEmbeddings(api_key=openai_api_key)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


# Vectorstore From Langchain

In [ ]:
vectorstore = Chroma(embedding_function=embeddings)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


In [ ]:
vectorstore.add_texts(texts)

['d646bbc8-a3d9-4c54-8d23-3d40c734e25e',
 '8a664ad0-8d97-4044-b486-2a4de8d8b4b7',
 '5694c8ff-92f4-4258-9bfa-7b605d6b2971',
 '89dd5a58-ab36-45f4-9ab0-287276a52e15',
 '2ba860eb-c83e-4e36-a924-6f13de2b8f00',
 '127c0f81-6a5f-4dce-ba72-dc8649c75fa1',
 '169d1dea-3d5a-4bb5-a49a-818e2fde7f2d',
 '24dbe132-7667-4867-8db5-1167515eb3af',
 'b9042700-add4-4214-905a-f718816d8cd0',
 'fa0003dd-31ec-42df-af7e-5b6b9c9d74c6',
 '49aa2c8d-f210-488f-a5e8-a037b6107e2c',
 '02800fd7-ce0e-4362-b6c7-bddda98647f8',
 '4dd61086-6051-4d88-a5be-9f9787d41930',
 '465058de-31ec-414d-95de-20d744164de3',
 '26f8b0f3-424f-471f-8d90-5c5825fbf2e6',
 'cb66f9ee-4adb-42eb-88ee-c3ea935b7131']

# Define LLM Chain And PromptTemplate

In [ ]:
# Define the prompt templates
combine_docs_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="Given the following context, answer the question.\nContext: {context}\nQuestion: {question}\nAnswer:"
)
question_gen_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="Given the following context, generate a detailed question.\nContext: {context}\nQuestion: {question}\nDetailed Question:"
)

# Define the LLM chains for combining documents and generating questions
llm_chain_for_combine = LLMChain(llm=llm, prompt=combine_docs_prompt)
question_gen_chain = LLMChain(llm=llm, prompt=question_gen_prompt)

# Define the StuffDocumentsChain
combine_docs_chain = StuffDocumentsChain(llm_chain=llm_chain_for_combine, document_variable_name="context")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [ ]:
# Define the conversational retrieval chain
chain = ConversationalRetrievalChain(
    combine_docs_chain=combine_docs_chain,
    question_generator=question_gen_chain,
    retriever=vectorstore.as_retriever()
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ConversationalRetrievalChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use create_history_aware_retriever together with create_retrieval_chain (see example in docstring) instead.
  warn_deprecated(


# Define Function For Query

In [ ]:
# chat_history = []
# def query_chain(query, chat_history):
#     docs = vectorstore.similarity_search(query, k=5)  # Adjust k as needed
#     result = chain({"question": query, "chat_history": chat_history, "contex": docs})
#     # chat_history.append((query, result['answer']))
#     return result['answer']

In [ ]:
chat_history = []
def query_chain(query, chat_history):
    docs = vectorstore.similarity_search(query, k=5)  # Adjust k as needed
    result = chain({"question": query, "chat_history": chat_history, "context": docs})
    # chat_history.append((query, result['output_text']))

    # Check if the response is complete or truncated
    while not result['answer'].strip().endswith(('.', '!', '?')):
        follow_up_query = "Can you continue the answer?"
        follow_up_response = chain({"question": follow_up_query, "chat_history": chat_history, "context": docs})
        result['answer'] += " " + follow_up_response['answer']
        # chat_history.append((follow_up_query, follow_up_response['answer']))

    return result['answer']

# Ask The Question And Get Answer

In [ ]:
query = "What is Inverted Top Hand Drill? Explain"
response = query_chain(query, chat_history)

print("Response:")
print(response)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Response:
 The Inverted Top Hand Drill is a batting drill that aims to help hitters who pull a lot of hard foul balls due to an overactive top hand. This drill isolates the bottom hand and teaches it to work together with the top hand. To execute this drill, the hitter sets up a batting tee for a pitch down the middle of the plate and addresses home plate as they would in a game. They then narrow their stance and move their front foot so their feet are square to home plate. The top hand is taken off the bat and turned upside down, with the thumbs from both hands touching. The back elbow is kept high and the top hand remains loosely on the bat, only pushing. The focus is on using the bottom hand to do most of the work and hit line drives to the back of the cage. This drill helps hitters to develop a better bat path and prevent the top hand from taking over too quickly. It also helps with weight transfer from the backside to the front side and keeping the front shoulder in during the swi

In [ ]:
query = "How to execute Separation Drill?"
response = query_chain(query, chat_history)

print("Response:")
print(response)

Response:
 To execute the Separation Drill, follow these steps:

1. Set up the batting tee for a pitch down the middle of the plate.
2. Start in your normal batting stance, facing home plate.
3. Lift your front knee and hold it parallel to the ground.
4. At the same time, take your bat from resting on your back shoulder and move your hands back away from your body.
5. Stride towards the pitch, keeping your head and body still.
6. As your front foot lands, make sure your weight is evenly distributed.
7. Hold this position for a second or two.
8. Repeat this drill, focusing on maintaining proper balance and separation between your hands and body. 


In [ ]:
query = "How to execute 45 angle drill?"
response = query_chain(query, chat_history)

print("Response:")
print(response)

Response:
 Here are the steps to execute the 45 degree angle drill:

1. Set up the batting tee: Place the batting tee on top of home plate and adjust its position to make the drill easier or more difficult.

2. Position your feet: Place your back foot in line with the angle of home plate from the outside corner to the point. This imaginary line is where your back foot should be. Your front foot should be square to your back foot, creating a 45 degree angle to home plate.

3. Get in an athletic stance: Make sure to have some flex in your front leg (about 60% of your body weight) as you start your swing. This will help you stay balanced and athletic throughout the drill.

4. Swing without striding: In this drill, you will not take a stride. Instead, you will widen your stance to find your athletic base and keep your front foot in place as you swing.

5. Focus on getting the bat in the zone early: The key to this drill is getting the bat into the hitting zone early and on plane with the p